# Simulation of a room following different routes

In this demonstration, we'll be simulating a room with a single exit. 
We'll place two distinct groups of agents in a designated zone within the room. 
Each group will be assigned a specific route to reach the exit: 
one group will follow the shortest path, while the other will take a longer detour.

To chart these paths, we'll use several waypoints, creating unique journeys for the agents to navigate.

## Configuring the Room Layout

For our simulation, we'll utilize a square-shaped room with dimensions of 20 meters by 20 meters. 
Inside, obstacles will be strategically placed to segment the room and guide both agent groups.

**Note** that the obstacles can not intersect the geometry. 

In [ ]:
from shapely import GeometryCollection, Polygon, to_wkt
import pathlib
import pandas as pd
import numpy as np
import jupedsim as jps
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Figure
import pedpy
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
%matplotlib inline

In [ ]:
complete_area = [
    (0, 0),
    (0, 20),
    (20, 20),
    (20, 0),
]
obstacles = [
    [
        (5, 0.01),
        (5, 18),
        (5.2, 18),
        (5.2, 0.01),
    ],
    [(15, 18), (15, 5), (7.2, 5), (7.2, 4.8), (15.2, 4.8), (15.2, 18)],
]

exit_polygon = [(18, 19), (20, 19), (20, 20), (18, 20)]
waypoints = [([3, 19], 1), ([7, 19], 2), ([7, 2.5], 2), ([17.5, 2.5], 2)]
distribution_polygon = Polygon([[0, 0],[5, 0],[5, 10],[0, 10]])
walkable_area = pedpy.WalkableArea(complete_area, obstacles)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_aspect('equal')
pedpy.plot_walkable_area(walkable_area=walkable_area, axes=ax)

for idx, (waypoint, distance) in enumerate(waypoints): 
    ax.plot(waypoint[0], waypoint[1], "ro")
    ax.annotate(f"WP {idx+1}", (waypoint[0], waypoint[1]), textcoords="offset points", xytext=(10,-15), ha='center')
    circle = Circle((waypoint[0], waypoint[1]), distance, fc='red', ec='red', alpha=0.1)
    ax.add_patch(circle)

x, y = Polygon(exit_polygon).exterior.xy
plt.fill(x, y, alpha=0.1, color='orange') 
centroid = Polygon(exit_polygon).centroid
plt.text(centroid.x, centroid.y, "Exit", ha='center', va='center', fontsize=8);

x, y = distribution_polygon.exterior.xy
plt.fill(x, y, alpha=0.1, color='blue') 
centroid = distribution_polygon.centroid
plt.text(centroid.x, centroid.y, "Start", ha='center', va='center', fontsize=10)

## Setting up the Operational Model
<a id="model"></a>
With our room geometry in place, the next step is to define the simulation object, the operational model and its corresponding parameters. In this demonstration, we'll use the "collision-free" model.

In [ ]:
trajectory_file = "trajectories.sqlite"
simulation = jps.Simulation(
        model=jps.VelocityModelParameters(a_ped=2.6, d_ped=0.1, d_wall=0.05),
        geometry=walkable_area.polygon,
        trajectory_writer=jps.SqliteTrajectoryWriter(
            output_file=pathlib.Path(trajectory_file)
        ),
    )

## Charting the Agent Pathways

With our foundational setup in place, it's time to outline the routes our agents will take. We've designated two distinct pathways:

- The first route is a direct path, guiding agents along the shortest distance to the exit.
- The second route, in contrast, takes agents on a more extended journey, guiding them along the longest distance to reach the same exit.

These variations in routing are designed to showcase how agents navigate and respond under different evacuation strategies.

In [ ]:
exit_id = simulation.add_exit_stage(exit_polygon)
waypoint_ids=[]
for (waypoint, distance) in waypoints:
     waypoint_ids.append(simulation.add_waypoint_stage(waypoint,distance))
          
long_journey = jps.JourneyDescription([*waypoint_ids[:], exit_id])
for idx, waypoint in enumerate(waypoint_ids):
    next_waypoint = exit_id if idx == len(waypoint_ids) - 1 else waypoint_ids[idx + 1]
    long_journey.set_transition_for_stage(waypoint, jps.Transition.create_fixed_transition(next_waypoint))


short_journey = jps.JourneyDescription([waypoint_ids[0], exit_id])
short_journey.set_transition_for_stage(waypoint_ids[0], jps.Transition.create_fixed_transition(exit_id))

long_journey_id = simulation.add_journey(long_journey)
short_journey_id = simulation.add_journey(short_journey)

## Allocation and Configuration of Agents

With our environment set up, it's time to introduce and configure the agents, utilizing the parameters we've previously discussed. We're going to place agents in two distinct groups, the proportion of which will be determined by the specified percentage parameter.

- The first group will be directed to take the longer route to the exit.
- Conversely, the second group will be guided along the shortest path to reach the exit.

By doing so, we aim to observe and analyze the behaviors and dynamics between these two groups under varying evacuation strategies.

In [ ]:
percentage = 50 # this we should change
agent_parameters = jps.VelocityModelAgentParameters()
agent_parameters.orientation = (0.0, 1.0)
agent_parameters.time_gap = 1
agent_parameters.v0 = 1.2
agent_parameters.radius = 0.2
agent_parameters.stage_id = waypoint_ids[0]
total_agents = 20
positions = jps.distribute_by_number(polygon=distribution_polygon,
            number_of_agents=total_agents,
            distance_to_agents=0.4,
            distance_to_polygon=0.7,
            seed=45131502,
        )

num_items = int(len(positions) * (percentage / 100.0)) 
agent_parameters.journey_id = short_journey_id
for new_pos in positions[:num_items]:  
    agent_parameters.position = new_pos
    simulation.add_agent(agent_parameters)

agent_parameters.journey_id = long_journey_id
for new_pos in positions[num_items:]:  
    agent_parameters.position = new_pos
    simulation.add_agent(agent_parameters)

## Launching the Simulation Run

Having  configured our environment, agents, and routes, we are now poised to set the simulation into motion. For the purposes of this demonstration, agent trajectories throughout the simulation will be systematically captured and stored within an SQLite database. This will allow for a detailed post-analysis of agent behaviors and movement patterns.

In [ ]:
while simulation.agent_count() > 0:
    simulation.iterate()

## Visualizing Agent Pathways

To gain insights into the movement patterns of our agents, we'll visualize their trajectories. Data for this endeavor will be pulled directly from the SQLite database we've previously populated. 

In [ ]:
import matplotlib.pyplot as plt
DUMMY_SPEED = -1000
def read_sqlite_file(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        walkable_area = (
            con.cursor().execute("select wkt from geometry").fetchone()[0]
        )
        return (
            pedpy.TrajectoryData(data=data, frame_rate=fps),
            pedpy.WalkableArea(walkable_area),
        )


def speed_to_color(speed, min_speed, max_speed):
    """Map a speed value to a color using a colormap."""
    normalized_speed = (speed - min_speed) / (max_speed - min_speed)
    r, g, b = plt.cm.jet_r(normalized_speed)[:3]
    return f"rgba({r*255:.0f}, {g*255:.0f}, {b*255:.0f}, 0.5)"

def get_line_color(disk_color):

    r, g, b, _ = [int(float(val)) for val in disk_color[5:-2].split(",")]
    brightness = (r * 299 + g * 587 + b * 114) / 1000
    return "black" if brightness > 127 else "white"  

def create_orientation_line(row, line_length=0.2, color='black'):
    end_x = row["x"] + line_length * row["ox"]
    end_y = row["y"] + line_length * row["oy"]

    orientation_line = go.layout.Shape(
        type="line",
        x0=row["x"],
        y0=row["y"],
        x1=end_x,
        y1=end_y,
        line=dict(color=color, width=3),
    )
    return orientation_line

def get_geometry_traces(area):
    geometry_traces = []
    x, y = area.exterior.xy
    geometry_traces.append(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
            showlegend=False,
            name="Exterior",
            hoverinfo="name",
        )
    )
    for inner in area.interiors:
        xi, yi = zip(*inner.coords[:])
        geometry_traces.append(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
                showlegend=False,
                name="Obstacle",
                hoverinfo="name",
            )
        )
    return geometry_traces

def get_colormap(frame_data, max_speed):
    """Utilize scatter plots with varying colors for each agent instead of individual shapes.

    This trace is only to incorporate a colorbar in the plot.
    """
    scatter_trace = go.Scatter(
        x=frame_data["x"],
        y=frame_data["y"],
        mode="markers",
        marker=dict(
            size=frame_data["radius"] * 2,
            color=frame_data["speed"],
            colorscale="Jet_r",
            colorbar=dict(title="Speed [m/s]"),
            cmin=0,
            cmax=max_speed,
        ),
        text=frame_data["speed"],
        showlegend=False,
        hoverinfo="none",
    )

    return [scatter_trace]

def get_shapes_for_frame(frame_data, min_speed, max_speed):
    def create_shape(row):
        hover_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            text=[f"ID: {row['id']}, Pos({row['x']:.2f},{row['y']:.2f})"],
            mode="markers",
            marker=dict(size=1, opacity=1),
            hoverinfo="text",
            showlegend=False,
        )
        if row["speed"] == DUMMY_SPEED:
            dummy_trace = go.Scatter(
                x=[row["x"]],
                y=[row["y"]],
                mode="markers",
                marker=dict(size=1, opacity=0),
                hoverinfo="none",
                showlegend=False,
            )
            return (
                go.layout.Shape(
                    type="circle",
                    xref="x",
                    yref="y",
                    x0=row["x"] - row["radius"],
                    y0=row["y"] - row["radius"],
                    x1=row["x"] + row["radius"],
                    y1=row["y"] + row["radius"],
                    line=dict(width=0),
                    fillcolor="rgba(255,255,255,0)",  # Transparent fill
                ),
                dummy_trace, create_orientation_line(row, color="rgba(255,255,255,0)")
            )
        color = speed_to_color(row["speed"], min_speed, max_speed)
        return (
            go.layout.Shape(
                type="circle",
                xref="x",
                yref="y",
                x0=row["x"] - row["radius"],
                y0=row["y"] - row["radius"],
                x1=row["x"] + row["radius"],
                y1=row["y"] + row["radius"],
                line_color=color,
                fillcolor=color,
            ),
            hover_trace, create_orientation_line(row,color=get_line_color(color))
        )

    results = frame_data.apply(create_shape, axis=1).tolist()
    shapes = [res[0] for res in results]
    hover_traces = [res[1] for res in results]
    arrows =  [res[2] for res in results]
    return shapes, hover_traces, arrows

def create_fig(
    initial_agent_count,
    initial_shapes,
    initial_arrows,
    initial_hover_trace,
    initial_scatter_trace,
    geometry_traces,
    frames,
    steps,
    area_bounds,
    width=800,
    height=800,
):
    """Creates a Plotly figure with animation capabilities.

    Returns:
        go.Figure: A Plotly figure with animation capabilities.
    """

    minx, miny, maxx, maxy = area_bounds
    title = f"<b>Number of Agents: {initial_agent_count}</b>"
    fig = go.Figure(
        data=geometry_traces
        + initial_scatter_trace
        #+ hover_traces
        + initial_hover_trace,
        frames=frames,
        layout=go.Layout(
            shapes=initial_shapes + initial_arrows, title=title, title_x=0.5
        ),
    )
    fig.update_layout(
        updatemenus=[_get_animation_controls()],
        sliders=[_get_slider_controls(steps)],
        autosize=False,
        width=width,
        height=height,
        xaxis=dict(range=[minx - 0.5, maxx + 0.5]),
        yaxis=dict(
            scaleanchor="x", scaleratio=1, range=[miny - 0.5, maxy + 0.5]
        ),
    )

    return fig


def _get_animation_controls():
    """Returns the animation control buttons for the figure."""
    return {
        "buttons": [
            {
                "args": [
                    None,
                    {
                        "frame": {"duration": 100, "redraw": True},
                        "fromcurrent": True,
                    },
                ],
                "label": "Play",
                "method": "animate",
            },
            {
                "args": [
                    [None],
                    {
                        "frame": {"duration": 0, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 0},
                    },
                ],
                "label": "Pause",
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top",
    }


def _get_slider_controls(steps):
    """Returns the slider controls for the figure."""
    return {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right",
        },
        "transition": {"duration": 100, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": steps,
    }

def animate(
    data: pedpy.TrajectoryData, area: pedpy.WalkableArea, *, every_nth_frame=50, width=800, height=800
):
    data_df = pedpy.compute_individual_speed(traj_data=data, frame_step=5)
    data_df = data_df.merge(data.data, on=["id", "frame"], how="left")
    data_df["radius"] = 0.2
    min_speed = data_df["speed"].min()
    max_speed = data_df["speed"].max()
    max_agents = data_df.groupby("frame").size().max()
    frames = []
    steps = []
    unique_frames = data_df["frame"].unique()
    selected_frames = unique_frames[::every_nth_frame]
    geometry_traces = get_geometry_traces(area.polygon)
    initial_frame_data = data_df[data_df["frame"] == data_df["frame"].min()]
    initial_agent_count = len(initial_frame_data)
    initial_shapes, initial_hover_trace, initial_arrows = get_shapes_for_frame(
        initial_frame_data, min_speed, max_speed
    )
    color_map_trace = get_colormap(
        initial_frame_data, max_speed
    )

    for frame_num in selected_frames[1:]:
        frame_data, agent_count = _get_processed_frame_data(
            data_df, frame_num, max_agents
        )
        shapes, hover_traces, arrows = get_shapes_for_frame(
            frame_data, min_speed, max_speed
        )
        frame = go.Frame(
            data=geometry_traces + hover_traces,
            name=str(frame_num),
            layout=go.Layout(
                shapes=shapes + arrows,
                title=f"<b>Number of Agents: {agent_count}</b>",
                title_x=0.5,
            ),
        )
        frames.append(frame)

        step = {
            "args": [
                [str(frame_num)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 500},
                },
            ],
            "label": str(frame_num),
            "method": "animate",
        }
        steps.append(step)

    return create_fig(
        initial_agent_count,
        initial_shapes,
        initial_arrows,
        initial_hover_trace,
        color_map_trace,
        geometry_traces,
        frames,
        steps,
        area.bounds,
        width=width,
        height=height,
    )


def _get_processed_frame_data(data_df, frame_num, max_agents):
    """Process frame data and ensure it matches the maximum agent count."""
    frame_data = data_df[data_df["frame"] == frame_num]
    agent_count = len(frame_data)
    dummy_agent_data = {"x": 0, "y": 0, "radius": 0, "speed": DUMMY_SPEED}
    while len(frame_data) < max_agents:
        dummy_df = pd.DataFrame([dummy_agent_data])
        frame_data = pd.concat([frame_data, dummy_df], ignore_index=True)
    return frame_data, agent_count

In [ ]:
agent_trajectories, walkable_area = read_sqlite_file(trajectory_file)
animate(agent_trajectories, walkable_area)

In [ ]:
t_evac = agent_trajectories.data['frame'].max()/agent_trajectories.frame_rate
print(f"Percentage {percentage}%, Evac time {t_evac} s")

## Troubleshooting

On certain occasions, improper configuration of the simulation, such as setting an agent's desired speed to 0, can cause the simulation loop to run indefinitely. If this happens, it's recommended to modify the loop condition. Instead of allowing it to run without constraints, consider limiting its duration using `simulation.iteration_count()`.